In [1]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.passes import FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout
from qiskit.converters import circuit_to_dag, dag_to_circuit
from utils import file_to_coupling_map, directory_to_circuits
import random
import time
import os
import numpy as np


In [2]:
seed = 42
random.seed(seed)

cmap_name= "heavy_hex_127.txt"
cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")
qc_name = "qft"
heuristic = "decay"
curr_seed = 42
qc_list = directory_to_circuits(f"circuits/{qc_name}")

pass_trials_list = sorted(set(np.logspace(0, 3, num=20, dtype=int)))


# CSV file to store the results
csv_path = f"results/trials_{qc_name}_20.csv"
# Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
with open(csv_path, "w") as f:
    f.write("swap_count,depth,runtime,num_qubits,heuristic,cmap,trials,seed\n")




In [3]:
# Run the circuits
for qc in qc_list:
    num_qubits = qc.num_qubits
    print(f"    Running {qc.name} ")
    # Run the circuits for different number of trials
    for pass_trials in pass_trials_list:
        print(f"Running {pass_trials} trials")
        # Create the sabre pass
        sabre_pass = SabreLayout(cmap, seed=curr_seed, max_iterations=4, swap_trials=pass_trials, layout_trials=pass_trials )
        dag = circuit_to_dag(qc)
        
        # Execute the pass manager
        start = time.time()
        dag_tr = sabre_pass.run(dag)
        end = time.time()
        # Collect the metrics for this trial
        runtime = end - start

        qc_tr = dag_to_circuit(dag_tr)
        swap_count = qc_tr.count_ops().get("swap", 0)
        depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

        # Write the results to the CSV file
        # Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
        with open(csv_path, "a") as f:
            f.write(f"{swap_count},{depth},{runtime},{num_qubits},{heuristic},{cmap_name},{pass_trials},{curr_seed}\n")
        print(f"            Swap count: {swap_count}, Depth: {depth}, Runtime: {runtime}")

    

    Running qft_10.qasm 
Running 1 trials
            Swap count: 57, Depth: 54, Runtime: 0.13671422004699707
Running 2 trials
            Swap count: 57, Depth: 54, Runtime: 0.08251595497131348
Running 4 trials
            Swap count: 57, Depth: 54, Runtime: 0.07835793495178223
Running 6 trials
            Swap count: 57, Depth: 54, Runtime: 0.07688021659851074
Running 8 trials
            Swap count: 56, Depth: 56, Runtime: 0.09735608100891113
Running 12 trials
            Swap count: 54, Depth: 49, Runtime: 0.1302788257598877
Running 18 trials
            Swap count: 54, Depth: 49, Runtime: 0.2534949779510498
Running 26 trials
            Swap count: 45, Depth: 54, Runtime: 0.4363059997558594
Running 37 trials
            Swap count: 45, Depth: 54, Runtime: 0.8164479732513428
Running 54 trials
            Swap count: 45, Depth: 54, Runtime: 1.3780620098114014
Running 78 trials
            Swap count: 45, Depth: 54, Runtime: 2.773362159729004
Running 112 trials
            Swap count